In [2]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.5/32.5 MB 94.7 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [geopandas]/4 [geopandas]
Note: you may need to restart the kernel to use updated packages.


#### Expo argile

In [ ]:
ExpoArgile = gpd.read_file("ExpoArgile_Fxx_4326.gpkg") 
ExpoArgile.head()

,mvt_id,ALEA,DPT,NIVEAU,geometry
0,NaN,Faible,29,1.0,"POLYGON ((-5.1318 48.45835, -5.1313 48.45831, ..."
1,NaN,Faible,29,1.0,"POLYGON ((-5.13253 48.4584, -5.13236 48.45839,..."
2,NaN,Faible,29,1.0,"POLYGON ((-5.12374 48.45835, -5.12374 48.45827..."
3,NaN,Faible,29,1.0,"POLYGON ((-5.12521 48.46142, -5.1251 48.4612, ..."
4,NaN,Faible,29,1.0,"POLYGON ((-5.10864 48.46153, -5.1087 48.46148,..."


#### Admi express

In [ ]:
import geopandas as gpd

gdf = gpd.read_file("ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20.gpkg") 
gdf.head()

/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read(
/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read(
/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read(
/opt/python/lib/python3.13/site-packages/pyogrio/raw.py:200: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read(
/opt/python/lib/python3.13/site-packages/pyogrio/geopandas.py:382: UserWarning: More than one layer found in 'ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20.gpkg': 'canton' (default), 'arrondissement', 'arrondissement_municipal', 'chef_lieu_d_arrondissement', 'chef_lieu_d_arrondissement_municipal', 'chef_lieu_de_canton', 'chef_lieu_de_collectivite_territoriale', 'chef_lieu_de_commune', 'chef_lieu_de_commune_associee_ou

,cleabs,nom_officiel,nom_officiel_en_majuscules,numero_du_canton,code_insee,codes_insee_des_arrondissements,code_insee_du_departement,code_insee_de_la_region,composition_du_canton,geometry
0,CANTON__0000000000008611,Migné-Auxances,MIGNE-AUXANCES,11,8611,863,86,75,Commune(s) entière(s),"MULTIPOLYGON (((485821.6 6621174.4, 485792.6 6..."
1,CANTON__0000000000006713,Reichshoffen,REICHSHOFFEN,13,6713,672,67,44,Fraction d'une commune et de commune(s) entièr...,"MULTIPOLYGON (((1038844.4 6869352.9, 1038825.6..."
2,CANTON__0000000000001810,La Guerche-sur-l'Aubois,LA GUERCHE-SUR-L'AUBOIS,10,1810,182,18,24,Fraction d'une commune et de commune(s) entièr...,"MULTIPOLYGON (((676322.6 6650709, 676352.3 665..."
3,CANTON__0000000000003916,Coteaux du Lizon,COTEAUX DU LIZON,16,3916,393,39,27,Commune(s) entière(s),"MULTIPOLYGON (((910159.4 6577137.8, 910114.2 6..."
4,CANTON__0000000000005213,Saint-Dizier-1,SAINT-DIZIER-1,13,5213,523,52,44,Fraction d'une commune et de commune(s) entièr...,"MULTIPOLYGON (((837589.8 6826780.5, 837574.4 6..."


### Jointure des datas

In [ ]:
!pip install dask_geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 49.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [dask_geopandas]m [dask_geopandas]


In [ ]:
import dask_geopandas as dgpd

gdf1 = gdf.to_crs(2154)  # polygones principaux
gdf2 = ExpoArgile.to_crs(2154)  # polygones à joindre

# Charger en Dask
dgdf1 = dgpd.from_geopandas(gdf1, npartitions=10)
dgdf2 = dgpd.from_geopandas(gdf2, npartitions=10)

# Jointure spatiale par intersection
joined2 = dgdf1.sjoin(dgdf2, predicate='intersects').compute()

# Vérification
joined2.head()



,cleabs,nom_officiel,nom_officiel_en_majuscules,numero_du_canton,code_insee,codes_insee_des_arrondissements,code_insee_du_departement,code_insee_de_la_region,composition_du_canton,geometry,index_right,mvt_id,ALEA,DPT,NIVEAU
5,CANTON__0000000000005026,Val-de-Saire,VAL-DE-SAIRE,26,5026,502,50,28,Commune(s) entière(s),"MULTIPOLYGON (((390956.7 6950881, 390961.5 695...",94878,NaN,Moyen,50,2.0
5,CANTON__0000000000005026,Val-de-Saire,VAL-DE-SAIRE,26,5026,502,50,28,Commune(s) entière(s),"MULTIPOLYGON (((390956.7 6950881, 390961.5 695...",94046,NaN,Faible,50,1.0
5,CANTON__0000000000005026,Val-de-Saire,VAL-DE-SAIRE,26,5026,502,50,28,Commune(s) entière(s),"MULTIPOLYGON (((390956.7 6950881, 390961.5 695...",94880,NaN,Faible,50,1.0
5,CANTON__0000000000005026,Val-de-Saire,VAL-DE-SAIRE,26,5026,502,50,28,Commune(s) entière(s),"MULTIPOLYGON (((390956.7 6950881, 390961.5 695...",94043,NaN,Moyen,50,2.0
5,CANTON__0000000000005026,Val-de-Saire,VAL-DE-SAIRE,26,5026,502,50,28,Commune(s) entière(s),"MULTIPOLYGON (((390956.7 6950881, 390961.5 695...",93187,NaN,Moyen,50,2.0


In [ ]:
# -------------------------------
# 1️⃣ Points volumineux
# -------------------------------
gdf_points = gpd.read_file("/home/onyxia/work/swi_brut_uniform_merged_2154.gpkg")
gdf_points = gdf_points[gdf_points["ym"] <= "2022-12"]
gdf_points = gdf_points[gdf_points["ym"] >= "2022-01"]

# Supprimer les colonnes 'index_right' et 'index' si elles existent
for col in ["index_right", "index"]:
    if col in gdf_points.columns:
        gdf_points = gdf_points.drop(columns=[col])

# -------------------------------
# 2️⃣ Polygones
# -------------------------------
gdf_polygons = joined2.copy()

# Supprimer les colonnes 'index_right' et 'index' si elles existent
for col in ["index_right", "index"]:
    if col in gdf_polygons.columns:
        gdf_polygons = gdf_polygons.drop(columns=[col])

In [ ]:
# S'assurer que l'index est propre
gdf_points = gdf_points.reset_index(drop=True)

# -------------------------------
# 2️⃣ Nettoyage DES POLYGONES
# -------------------------------
for col in ["index_right", "index_left", "index"]:
    if col in gdf_polygons.columns:
        gdf_polygons = gdf_polygons.drop(columns=[col])

# Index propre + id explicite
gdf_polygons = gdf_polygons.reset_index(drop=True)
gdf_polygons["polygon_id"] = gdf_polygons.index.astype("int64")

# -------------------------------
# 4️⃣ JOINTURE SPATIALE (POINT → POLYGONE)
# -------------------------------
points_with_polygons = gpd.sjoin(
    gdf_points,
    gdf_polygons,
    how="inner",          # garde TOUS les points
    predicate="within"   # point dans polygone
)

points_with_polygons.head()


In [ ]:
import duckdb

# ================================
# 1️⃣ Connexion DuckDB
# ================================
con = duckdb.connect()

# (optionnel mais recommandé pour gros volumes)
con.execute("PRAGMA memory_limit='8GB';")
con.execute("PRAGMA threads=4;")

# ================================
# 2️⃣ Enregistrer la GeoDataFrame
# ================================
con.register("points_with_polygons", points_with_polygons)

# ================================
# 3️⃣ Export en GeoPackage (GPKG)
# ================================
con.execute("""
    COPY (
        SELECT *
        FROM points_with_polygons
    )
    TO '/home/onyxia/work/data_22.gpkg'
    WITH (
        FORMAT GDAL,
        DRIVER 'GPKG',
        LAYER 'points'
    )
""")

print("✅ Export terminé : points_with_polygons.gpkg")
